# Deep Learning for Predictive Maintenance

This notebook aims at building a Long Short Time Memory [(LSTM)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) network for the scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict remaining useful life of aircraft engines. The training, test and ground truth datasets can be downloaded from azure storage as
* http://azuremlsamples.azureml.net/templatedata/PM_train.txt 
* http://azuremlsamples.azureml.net/templatedata/PM_test.txt 
* http://azuremlsamples.azureml.net/templatedata/PM_truth.txt 

The data needs to be parsed from the above text files . Here we use already parsed files.

In [1]:
!pip install keras
!pip install azure-storage

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from azure.storage.blob import BlockBlobService
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

Using TensorFlow backend.


In [3]:
azure_storage_account_name = "deeplearningcntkfb"
azure_storage_account_key = "ZeK8+b+SsNQFq5ynNu06cA31rCZYBCaBD9/UYh+zSAn2pRAY6KG3c+TWjlthE19h4JiZd0QT5g8V9KvG+G76lA=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("You must provide a name and key for an Azure Storage account")

In [4]:
# copy from Azure blob
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
containercontents = blob_service.list_blobs('deeplearning')
blob_service.get_blob_to_path('deeplearning','training_data.csv','/home/nbuser/training_data.csv')
blob_service.get_blob_to_path('deeplearning','test_data.csv','/home/nbuser/test_data.csv')
blob_service.get_blob_to_path('deeplearning','ground_truth.csv','/home/nbuser/ground_truth.csv')

In [28]:
# read data
train_df = pd.read_csv('training_data.csv')
test_df = pd.read_csv('test_data.csv')
truth_df = pd.read_csv('ground_truth.csv')

In [29]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


First step is to generate labels for the training data which are RUL, label1 and label2.

In [30]:
# generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [31]:
# generate label columns  for training data
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
#train_df['label2'] = np.where(train_df['RUL'] <= w0, 2, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0,0
5,1,6,-0.0043,-0.0001,100.0,518.67,642.10,1584.47,1398.37,14.62,...,8.4108,0.03,391,2388,100.0,38.98,23.3669,186,0,0
6,1,7,0.0010,0.0001,100.0,518.67,642.48,1592.32,1397.77,14.62,...,8.3974,0.03,392,2388,100.0,39.10,23.3774,185,0,0
7,1,8,-0.0034,0.0003,100.0,518.67,642.56,1582.96,1400.97,14.62,...,8.4076,0.03,391,2388,100.0,38.97,23.3106,184,0,0
8,1,9,0.0008,0.0001,100.0,518.67,642.12,1590.98,1394.80,14.62,...,8.3728,0.03,392,2388,100.0,39.05,23.4066,183,0,0
9,1,10,-0.0033,0.0001,100.0,518.67,641.71,1591.24,1400.46,14.62,...,8.4286,0.03,393,2388,100.0,38.95,23.4694,182,0,0


Here, we normalize the sensor columns in the training data. In the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) , cycle column is also normalized and used for training so we will also include the cycle column.

In [32]:
# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.01108


Next, we prepare the test data. We first normalize the test data using the parameters from the MinMax normalization applied on the training data. 

In [33]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108


Next, we use the ground truth dataset to generate labels for the test data.

In [34]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

In [35]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
test_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm,RUL
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.000000,142
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.002770,141
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.005540,140
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.008310,139
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.011080,138
5,1,6,0.568966,0.750000,0.0,0.0,0.271084,0.176150,0.217421,0.0,...,0.142017,0.380531,0.0,0.333333,0.0,0.0,0.596899,0.624827,0.013850,137
6,1,7,0.500000,0.666667,0.0,0.0,0.271084,0.268149,0.381330,0.0,...,0.180772,0.255868,0.0,0.250000,0.0,0.0,0.550388,0.691798,0.016620,136
7,1,8,0.534483,0.500000,0.0,0.0,0.400602,0.214737,0.314652,0.0,...,0.134121,0.370912,0.0,0.416667,0.0,0.0,0.705426,0.591273,0.019391,135
8,1,9,0.293103,0.500000,0.0,0.0,0.201807,0.485066,0.506921,0.0,...,0.176540,0.424779,0.0,0.250000,0.0,0.0,0.744186,0.770367,0.022161,134
9,1,10,0.356322,0.416667,0.0,0.0,0.259036,0.309789,0.276671,0.0,...,0.176179,0.324740,0.0,0.250000,0.0,0.0,0.565891,0.673571,0.024931,133


In [36]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
#test_df['label2'] = np.where(test_df['RUL'] <= w0, 2, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.000000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.002770,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.005540,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.008310,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.011080,138,0,0
5,1,6,0.568966,0.750000,0.0,0.0,0.271084,0.176150,0.217421,0.0,...,0.0,0.333333,0.0,0.0,0.596899,0.624827,0.013850,137,0,0
6,1,7,0.500000,0.666667,0.0,0.0,0.271084,0.268149,0.381330,0.0,...,0.0,0.250000,0.0,0.0,0.550388,0.691798,0.016620,136,0,0
7,1,8,0.534483,0.500000,0.0,0.0,0.400602,0.214737,0.314652,0.0,...,0.0,0.416667,0.0,0.0,0.705426,0.591273,0.019391,135,0,0
8,1,9,0.293103,0.500000,0.0,0.0,0.201807,0.485066,0.506921,0.0,...,0.0,0.250000,0.0,0.0,0.744186,0.770367,0.022161,134,0,0
9,1,10,0.356322,0.416667,0.0,0.0,0.259036,0.309789,0.276671,0.0,...,0.0,0.250000,0.0,0.0,0.565891,0.673571,0.024931,133,0,0


In the rest of the notebook, we train an LSTM model that we will compare to the results in [Predictive Maintenance Template Step 2B of 3](https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-2B-of-3-train-and-evaluate-binary-classification-models-2) where a series of machine learning models are used to train and evaluate the binary classification model that uses column "label1" as the label.

## LSTMs to predict failure
When using LSTMs in the time-series domain, one important parameter to pick is the sequence length which is the window for LSTMs to look back. This may be viewed as similar to picking window_size = 5 cycles for calculating the rolling features in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) which are rolling mean and   rolling standard deviation for 21 sensor values. The idea of using LSTMs is to let the model extract abstract features out of the sequence of sensor values in the window rather than engineering those manually. The expectation is that if there is a pattern in these sensor values within the window prior to failure, the pattern should be encoded by the LSTM.

In [37]:
sequence_length = 5

[Keras LSTM](https://keras.io/layers/recurrent/) layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step. 

In [38]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [39]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

In [40]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

In [41]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen))
seq_array.shape

(20131, 5, 25)

In [43]:
# take a peak at the first 2 (samples, time steps, features)
seq_array[0:2]

array([[[ 0.45977011,  0.16666667,  0.        ,  0.        ,  0.        ,
          0.18373494,  0.40680183,  0.30975692,  0.        ,  1.        ,
          0.72624799,  0.24242424,  0.109755  ,  0.        ,  0.36904762,
          0.63326226,  0.20588235,  0.1996078 ,  0.36398615,  0.        ,
          0.33333333,  0.        ,  0.        ,  0.71317829,  0.7246617 ],
        [ 0.6091954 ,  0.25      ,  0.        ,  0.00277008,  0.        ,
          0.28313253,  0.4530194 ,  0.35263336,  0.        ,  1.        ,
          0.62801932,  0.21212121,  0.1002423 ,  0.        ,  0.38095238,
          0.76545842,  0.27941176,  0.1628135 ,  0.41131204,  0.        ,
          0.33333333,  0.        ,  0.        ,  0.66666667,  0.73101353],
        [ 0.25287356,  0.75      ,  0.        ,  0.00554017,  0.        ,
          0.34337349,  0.36952256,  0.37052667,  0.        ,  1.        ,
          0.71014493,  0.27272727,  0.14004308,  0.        ,  0.25      ,
          0.79530917,  0.22058824,  

In [44]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    #yield data_array[seq_length:num_elements, :]
    return data_array[seq_length:num_elements, :]

In [45]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen)
label_array.shape

(20131, 1)

Next, we build a network. The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

In [46]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
 np.random.seed(1234) # for reproducibility 

In [48]:
# fit the network
model.fit(seq_array, label_array, epochs=5, batch_size = 200, validation_split=0.05, verbose=2)

Train on 19124 samples, validate on 1007 samples
Epoch 1/5
3s - loss: 0.2688 - acc: 0.8961 - val_loss: 0.1498 - val_acc: 0.9404
Epoch 2/5
3s - loss: 0.1194 - acc: 0.9506 - val_loss: 0.1221 - val_acc: 0.9454
Epoch 3/5
3s - loss: 0.1090 - acc: 0.9551 - val_loss: 0.1056 - val_acc: 0.9513
Epoch 4/5
3s - loss: 0.1040 - acc: 0.9559 - val_loss: 0.1162 - val_acc: 0.9583
Epoch 5/5
3s - loss: 0.1010 - acc: 0.9590 - val_loss: 0.0885 - val_acc: 0.9553


In [1]:
print(model.summary())

NameError: name 'model' is not defined

In [49]:
# training metrics
#print(model.metrics_names)
scores = model.evaluate(seq_array, label_array, verbose=2)
print('Accurracy: {}'.format(scores[1]))

Accurracy: 0.9549947841637276


In [50]:
# make predictions and compute confusion matrix
y_pred = model.predict_classes(seq_array)
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

20064/20131 [============================>.] - ETA: 0s 

array([[16517,   514],
       [  392,  2708]])

In [51]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  0.840471756673 
 recall =  0.873548387097


Next, we look at the performance on the test data. In the [Predictive Maintenance Template Step 1 of 3](https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-1-of-3-data-preparation-and-feature-engineering-2), only the last cycle data for each engine id in the test data is kept for testing purposes. In order to compare the results to the template, we pick the last sequence for each id in the test data.

In [52]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique()]
seq_array_test_last = np.asarray(seq_array_test_last)
seq_array_test_last.shape

(100, 5, 25)

In [53]:
# take a peak at the first 2 (samples, time steps, features)
seq_array_test_last[0:2]

array([[[ 0.45977011,  0.58333333,  0.        ,  0.07202216,  0.        ,
          0.26204819,  0.34030957,  0.30486158,  0.        ,  1.        ,
          0.72463768,  0.28787879,  0.10935116,  0.        ,  0.29166667,
          0.6673774 ,  0.20588235,  0.14088141,  0.4790304 ,  0.        ,
          0.33333333,  0.        ,  0.        ,  0.56589147,  0.68889809],
        [ 0.62643678,  0.91666667,  0.        ,  0.07479224,  0.        ,
          0.21686747,  0.5059952 ,  0.32140446,  0.        ,  1.        ,
          0.59742351,  0.25757576,  0.15179934,  0.        ,  0.11904762,
          0.67164179,  0.27941176,  0.18035917,  0.46979608,  0.        ,
          0.33333333,  0.        ,  0.        ,  0.53488372,  0.62966031],
        [ 0.58045977,  0.58333333,  0.        ,  0.07756233,  0.        ,
          0.22289157,  0.35120994,  0.26772451,  0.        ,  1.        ,
          0.69243156,  0.27272727,  0.10939603,  0.        ,  0.33928571,
          0.78891258,  0.27941176,  

Similarly, we pick the labels.

In [54]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1).values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1)
label_array_test_last.shape

(100, 1)

In [55]:
# test metrics
#print(model.metrics_names)
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

Accurracy: 0.94


In [56]:
# make predictions and compute confusion matrix
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

 32/100 [========>.....................] - ETA: 0sConfusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[74,  1],
       [ 5, 20]])

In [63]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

Precision:  0.952380952381 
 Recall:  0.8 
 F1-score: 0.869565217391


In [65]:
results_df = pd.DataFrame([[scores_test[1],precision_test,recall_test,f1_test],
                          [0.94, 0.952381, 0.8, 0.869565]],
                         columns = ['Accuracy', 'Precision', 'Recall', 'F1-score'],
                         index = ['LSTM',
                                 'Template Best Model'])
results_df

,Accuracy,Precision,Recall,F1-score
LSTM,0.94,0.952381,0.8,0.869565
Template Best Model,0.94,0.952381,0.8,0.869565


Comparing the above test results to the predictive maintenance template, we see that the results are same with each other. This is due to the fact that the test set used is very small (100) samples so there is not enough samples for different algorithms to show variation in the results. Deep learning models are known to perform superior with large datasets so for a more fair comparison larger datasets should be used.